In [1]:
import pandas as pd
import psycopg2 as psy
import os
from dotenv import load_dotenv

try:

    load_dotenv() # - carregando as variáveis de ambiente para que possam ser utilizadas
    
    conexao = psy.connect(
        dbname = os.getenv("DB_NAME"),
        user = os.getenv("USER"),
        password = os.getenv("PASSWORD"),
        host = os.getenv("HOST"),
        port = os.getenv("PORT")
    ) # - estabelecendo a conexão com o banco de dados

    cursor = conexao.cursor() 

    #execução do comando para trazer os dados da tabela original
    cursor.execute(
        '''SELECT * FROM usuario;'''
    )

    #criando um dataframe apartir dos dados capturados
    df = pd.DataFrame(cursor.fetchall(), columns=[item[0] for item in cursor.description])
    display(df)#mostrando o dataframe


    cursor.close()
    conexao.close()

except Exception as e:
    print(f"Erro ao conectar com o banco: "+str(e)) # - tratando exceção, (caso ocorra)

,id,fk_fabrica,id_gerente,email,senha,nome,genero,tipo_acesso,desc_tipoacesso,status,data_criacao,data_nascimento,cargo
0,1ad34692-83fa-46a3-acd3-ebcd87fea9ed,1,None,gerente.seara@seara.com.br,$2a$15$mABY7ntafExp1hTioSpzB.5EQBRlfIjuXA4Sc5F...,João Mendes,masc,1,Remover ábaco,True,2025-10-27 11:43:22.777548,1980-05-15,Gerente
1,1128f100-8290-4bce-976e-fc750d598b4a,1,None,supervisor.seara@seara.com.br,$2a$15$mABY7ntafExp1hTioSpzB.5EQBRlfIjuXA4Sc5F...,Ana Costa,fem,2,leitura e revisão de dados,True,2025-10-27 11:43:22.777548,1985-08-22,Gerente
2,d63ac3c7-b7f4-4a63-92f1-c5db2db47906,2,None,gerente.sadia@sadia.com.br,$2a$15$mABY7ntafExp1hTioSpzB.5EQBRlfIjuXA4Sc5F...,Pedro Oliveira,masc,1,Acesso total,True,2025-10-27 11:43:22.777548,1978-12-10,Gerente
3,2e50af4b-6ee8-417d-91d0-71ed47e5b949,3,None,gerente.jbs@jbs.com.br,$2a$15$mABY7ntafExp1hTioSpzB.5EQBRlfIjuXA4Sc5F...,Fernanda Lima,fem,1,Acesso total,True,2025-10-27 11:43:22.777548,1982-03-30,Gerente
4,e0d071ca-4037-4efe-8841-8589dac60aa8,4,None,supervisor.lins@jbs.com.br,$2a$15$mABY7ntafExp1hTioSpzB.5EQBRlfIjuXA4Sc5F...,Ricardo Souza,masc,2,Acesso limitado,True,2025-10-27 11:43:22.777548,1990-07-18,Gerente
5,a84e22da-7491-46af-91cd-ce05639d5f07,1,1ad34692-83fa-46a3-acd3-ebcd87fea9ed,usuariocomum.seara@seara.com.br,Seara12345,João Mendes,masc,1,Remover ábaco,True,2025-10-27 11:47:12.265229,1980-05-15,Usuário comum
6,af9bdf12-7b74-4ce7-8eb5-4f3575b88602,1,1128f100-8290-4bce-976e-fc750d598b4a,usuariocomum1.seara@seara.com.br,Seara12345,Ana Costa,fem,2,leitura e revisão de dados,True,2025-10-27 11:47:12.265229,1985-08-22,Usuário comum
7,e4c1b2fb-9712-4c28-91df-605c2369d7f3,2,d63ac3c7-b7f4-4a63-92f1-c5db2db47906,usuariocomum.sadia@sadia.com.br,Sadia12345,Pedro Oliveira,masc,1,Acesso total,True,2025-10-27 11:47:12.265229,1978-12-10,Usuário comum
8,738db049-d6b2-4908-9cb7-b76756130393,3,2e50af4b-6ee8-417d-91d0-71ed47e5b949,usuariocomum.jbs@jbs.com.br,Jbs12345,Fernanda Lima,fem,1,Acesso total,True,2025-10-27 11:47:12.265229,1982-03-30,Usuário comum
9,e08814c6-e30b-45a0-9c54-22d5f5528818,4,e0d071ca-4037-4efe-8841-8589dac60aa8,usuariocomum.lins@jbs.com.br,Jbs12345,Ricardo Souza,masc,2,Acesso limitado,True,2025-10-27 11:47:12.265229,1990-07-18,Usuário comum


In [2]:
#Pseudonimizando as colunas da tabela
df_user_pseudo = df.copy() # - criando uma cópia do dataframe para pseudonimizar de forma eficiente

#Excluindo campos do dataframe que não serão necessários
df_user_pseudo = df_user_pseudo.drop(columns="tipo_acesso")


#Pseudonimização da coluna de id 
df_user_pseudo['id'] = df_user_pseudo['id'].apply(lambda id: f'{"*" * (len(str(id))-1)}{str(id)[-10:]}')
#Pseudonimização da coluna de fk_fabrica
df_user_pseudo['fk_fabrica'] = ['FK' for fk in df_user_pseudo['fk_fabrica']]
#Pseudonimização da coluna de id_gerente
df_user_pseudo['id_gerente'] = ['ID' for id in df_user_pseudo['id_gerente']]
#Pseudonimização da coluna de email
df_user_pseudo['email'] = df_user_pseudo['email'].apply(lambda email: email[:3] + f"{'*' * (email.find('@') -3)}"+email[email.find('@'):])
#Pseudonimização da coluna de senha - criptografia
#Pseudonimização da coluna de nome
df_user_pseudo['nome'] = ['User' + str(cont).zfill(3) for cont in range(len(df_user_pseudo))]
#Pseudonimização da coluna de genero - Sem necessidade
#Pseudonimização da coluna de desc_tipoacesso
df_user_pseudo['desc_tipoacesso'] = df_user_pseudo['desc_tipoacesso'].apply(lambda desc: desc[:1]+f"{'*' * (len(desc[:]) -1)}")
#Pseudonimização da coluna de status
df_user_pseudo['status'] = df_user_pseudo['status'].apply(lambda status: str(status)[:1]+f'{"*" * (len(str(status))-1)}')
#Pseudonimização da coluna de data_criacao
df_user_pseudo['data_criacao'] = df_user_pseudo['data_criacao'].apply(lambda data: f'{str(data)[:4]}/**/** - {"*" * (len(str(data)) -4)}')
#Pseudonimização da coluna de data_nascimento
df_user_pseudo['data_nascimento'] = df_user_pseudo['data_nascimento'].apply(lambda data: f'{str(data)[:4]}/**/**')
#Pseudonimização da coluna de cargo
df_user_pseudo['cargo'] = df_user_pseudo['cargo'].apply(lambda cargo: cargo[:2]+f'{"*" * (len(cargo)-2)}')


display(df_user_pseudo)



,id,fk_fabrica,id_gerente,email,senha,nome,genero,desc_tipoacesso,status,data_criacao,data_nascimento,cargo
0,***********************************cd87fea9ed,FK,ID,ger**********@seara.com.br,$2a$15$mABY7ntafExp1hTioSpzB.5EQBRlfIjuXA4Sc5F...,User000,masc,R************,T***,2025/**/** - **********************,1980/**/**,Ge*****
1,***********************************750d598b4a,FK,ID,sup*************@seara.com.br,$2a$15$mABY7ntafExp1hTioSpzB.5EQBRlfIjuXA4Sc5F...,User001,fem,l*************************,T***,2025/**/** - **********************,1985/**/**,Ge*****
2,***********************************db2db47906,FK,ID,ger**********@sadia.com.br,$2a$15$mABY7ntafExp1hTioSpzB.5EQBRlfIjuXA4Sc5F...,User002,masc,A***********,T***,2025/**/** - **********************,1978/**/**,Ge*****
3,***********************************ed47e5b949,FK,ID,ger********@jbs.com.br,$2a$15$mABY7ntafExp1hTioSpzB.5EQBRlfIjuXA4Sc5F...,User003,fem,A***********,T***,2025/**/** - **********************,1982/**/**,Ge*****
4,***********************************89dac60aa8,FK,ID,sup************@jbs.com.br,$2a$15$mABY7ntafExp1hTioSpzB.5EQBRlfIjuXA4Sc5F...,User004,masc,A**************,T***,2025/**/** - **********************,1990/**/**,Ge*****
5,***********************************05639d5f07,FK,ID,usu***************@seara.com.br,Seara12345,User005,masc,R************,T***,2025/**/** - **********************,1980/**/**,Us***********
6,***********************************3575b88602,FK,ID,usu****************@seara.com.br,Seara12345,User006,fem,l*************************,T***,2025/**/** - **********************,1985/**/**,Us***********
7,***********************************5c2369d7f3,FK,ID,usu***************@sadia.com.br,Sadia12345,User007,masc,A***********,T***,2025/**/** - **********************,1978/**/**,Us***********
8,***********************************6756130393,FK,ID,usu*************@jbs.com.br,Jbs12345,User008,fem,A***********,T***,2025/**/** - **********************,1982/**/**,Us***********
9,***********************************d5f5528818,FK,ID,usu**************@jbs.com.br,Jbs12345,User009,masc,A**************,T***,2025/**/** - **********************,1990/**/**,Us***********


In [3]:
#Renomeando as colunas do dataframe
df_user_pseudo.rename(columns={
    "id":"id",
    "fk_fabrica":"fk_fabrica_pseudo",
    "id_gerente":"id_gerente_pseudo",
    "email": "email_pseudo",
    "senha": "senha_criptografada",
    "nome":"nome_pseudo",
    "genero":"genero",
    "desc_tipoacesso":"desc_tipoacesso_pseudo",
    "status":"status_pseudo",
    "data_criacao":"data_criacao_pseudo",
    "data_nascimento":"data_nascimento_pseudo",
    "cargo" : "cargo_pseudo"
    
}, inplace=True)

In [4]:
#Tranferindo todas as informações do dataframe pseudonimizado para o banco de dados 


try:
    
    conexao = psy.connect(
        dbname = os.getenv("DB_PSEUDO"),
        user = os.getenv("USER"),
        password = os.getenv("PASSWORD"),
        host = os.getenv("HOST"),
        port = os.getenv("PORT")
    ) # - estabelecendo a conexão com o banco de dados

    cursor = conexao.cursor()

    #Execução do comando para trazer os dados da tabela original
    query = """
    DELETE FROM usuario_pseudo;
    """ # - comando que vai ser aplicado

    valores = [tuple(x) for x in df_user_pseudo.to_numpy()] # - transformando cada linha do dataframe em uma tupla

    for c in valores:
        cursor.execute(query, c) # - executando o comando no banco de dados

    conexao.commit() # - salvando as alterações


    cursor.close() # - fechando conexão 
    conexao.close() # - fechando conexão 

except Exception as e:
    print(f"Erro ao conectar com o banco: "+str(e)) # - tratando exceção, (caso ocorra)

In [5]:
#Tranferindo todas as informações do dataframe pseudonimizado para o banco de dados 


try:
    
    conexao = psy.connect(
        dbname = os.getenv("DB_PSEUDO"),
        user = os.getenv("USER"),
        password = os.getenv("PASSWORD"),
        host = os.getenv("HOST"),
        port = os.getenv("PORT")
    ) # - estabelecendo a conexão com o banco de dados

    cursor = conexao.cursor()

    #Execução do comando para trazer os dados da tabela original
    query = """
    INSERT INTO usuario_pseudo (id, fk_fabrica_pseudo, id_gerente_pseudo, email_pseudo, senha_criptografada, nome_pseudo, genero, desc_tipoacesso_pseudo, status_pseudo, data_criacao_pseudo, data_nascimento_pseudo, cargo_pseudo)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) """ # - comando que vai ser aplicado

    valores = [tuple(x) for x in df_user_pseudo.to_numpy()] # - transformando cada linha do dataframe em uma tupla

    for c in valores:
        cursor.execute(query, c) # - executando o comando no banco de dados

    conexao.commit() # - salvando as alterações


    cursor.close() # - fechando conexão 
    conexao.close() # - fechando conexão 

except Exception as e:
    print(f"Erro ao conectar com o banco: "+str(e)) # - tratando exceção, (caso ocorra)

In [6]:
#Tranferindo as informações originais e pseudonimizadas para os arquivos csv
df.to_csv('original_user.csv')
df_user_pseudo.to_csv('pseudo_user.csv')